In [18]:
import pandas as pd 
import pyarrow.parquet as pq
import pyarrow as pa

In [2]:
df = pd.read_csv('ADMISSIONS.csv')
df.iloc[:6,:]

,row_id,subject_id,hadm_id,admittime,dischtime,deathtime,admission_type,admission_location,discharge_location,insurance,language,religion,marital_status,ethnicity,edregtime,edouttime,diagnosis,hospital_expire_flag,has_chartevents_data
0,12258,10006,142345,2164-10-23 21:09:00,2164-11-01 17:15:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,HOME HEALTH CARE,Medicare,NaN,CATHOLIC,SEPARATED,BLACK/AFRICAN AMERICAN,2164-10-23 16:43:00,2164-10-23 23:00:00,SEPSIS,0,1
1,12263,10011,105331,2126-08-14 22:32:00,2126-08-28 18:59:00,2126-08-28 18:59:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Private,NaN,CATHOLIC,SINGLE,UNKNOWN/NOT SPECIFIED,NaN,NaN,HEPATITIS B,1,1
2,12265,10013,165520,2125-10-04 23:36:00,2125-10-07 15:13:00,2125-10-07 15:13:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,NaN,CATHOLIC,NaN,UNKNOWN/NOT SPECIFIED,NaN,NaN,SEPSIS,1,1
3,12269,10017,199207,2149-05-26 17:19:00,2149-06-03 18:42:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,SNF,Medicare,NaN,CATHOLIC,DIVORCED,WHITE,2149-05-26 12:08:00,2149-05-26 19:45:00,HUMERAL FRACTURE,0,1
4,12270,10019,177759,2163-05-14 20:43:00,2163-05-15 12:00:00,2163-05-15 12:00:00,EMERGENCY,TRANSFER FROM HOSP/EXTRAM,DEAD/EXPIRED,Medicare,NaN,CATHOLIC,DIVORCED,WHITE,NaN,NaN,ALCOHOLIC HEPATITIS,1,1
5,12277,10026,103770,2195-05-17 07:39:00,2195-05-24 11:45:00,NaN,EMERGENCY,EMERGENCY ROOM ADMIT,REHAB/DISTINCT PART HOSP,Medicare,NaN,OTHER,NaN,WHITE,2195-05-17 01:49:00,2195-05-17 08:29:00,STROKE/TIA,0,1


In [ ]:
import pandas as pd
import pyarrow.parquet as pq
import pyarrow as pa

# Read CSV
df = pd.read_csv('ADMISSIONS.csv')

# Convert datetime column to string in Hive-compatible format
df['admittime'] = pd.to_datetime(df['admittime']).dt.strftime('%Y-%m-%d %H:%M:%S')#'dischtime', 'deathtime', 'edregtime', 'edouttime'
df['dischtime'] = pd.to_datetime(df['dischtime']).dt.strftime('%Y-%m-%d %H:%M:%S')
df['deathtime'] = pd.to_datetime(df['deathtime']).dt.strftime('%Y-%m-%d %H:%M:%S')
df['edregtime'] = pd.to_datetime(df['edregtime']).dt.strftime('%Y-%m-%d %H:%M:%S')
df['edouttime'] = pd.to_datetime(df['edouttime']).dt.strftime('%Y-%m-%d %H:%M:%S')
# 3. Categorical/string columns
cat_cols = [
    'admission_type', 'admission_location', 'discharge_location',
    'insurance', 'language', 'religion', 'marital_status',
    'ethnicity', 'diagnosis'
]
df[cat_cols] = df[cat_cols].astype('category')

# 4. Boolean/flag columns
bool_cols = ['hospital_expire_flag', 'has_chartevents_data']
df[bool_cols] = df[bool_cols].astype('boolean')


In [ ]:
# 2. language: Impute with "Unknown"
df['language'] = df['language'].cat.add_categories('Unknown')
df['language']=df['language'].fillna('Unknown')

# 3. religion: Impute with "Unknown"
df['religion'] = df['religion'].cat.add_categories('Unknown')
df['religion']=df['religion'].fillna('Unknown')

# 4. marital_status: Impute with "Unknown"
df['marital_status'] = df['marital_status'].cat.add_categories('Unknown')
df['marital_status']=df['marital_status'].fillna('Unknown')

# 5. edregtime and edouttime: Impute with sentinel datetime
df['edregtime']=df['edregtime'].fillna('9999-12-31 00:00:00')
df['edouttime']=df['edouttime'].fillna('9999-12-31 00:00:00')


In [ ]:
print("\nNaN counts after handling:")
print(df.isna().sum())

In [19]:
# Convert to Arrow table and write to Parquet
table = pa.Table.from_pandas(df)
#

In [20]:
print(table.schema)

row_id: int64
subject_id: int64
hadm_id: int64
admittime: string
dischtime: string
deathtime: string
admission_type: dictionary<values=string, indices=int8, ordered=0>
admission_location: dictionary<values=string, indices=int8, ordered=0>
discharge_location: dictionary<values=string, indices=int8, ordered=0>
insurance: dictionary<values=string, indices=int8, ordered=0>
language: dictionary<values=string, indices=int8, ordered=0>
religion: dictionary<values=string, indices=int8, ordered=0>
marital_status: dictionary<values=string, indices=int8, ordered=0>
ethnicity: dictionary<values=string, indices=int8, ordered=0>
edregtime: string
edouttime: string
diagnosis: dictionary<values=string, indices=int8, ordered=0>
hospital_expire_flag: bool
has_chartevents_data: bool
-- schema metadata --
pandas: '{"index_columns": [{"kind": "range", "name": null, "start": 0, "' + 2914
